In [1]:
import json
import math
import os

from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv()

False

In [3]:
def calculate_circle_area(radius):
    area = math.pi * radius * radius
    print(f"radius: {radius}, area: {area}")
    return math.pi * radius * radius




In [4]:
function_schema = [
    {
        "name": "func_1",
        "description": "This is what the function does and this is when the function should be called.",
        "parameters": {
            "type": "object",
            "properties": {
                "argument_1": {"type": "data_type", "description": "What argument_1 is and its description"}
            },
            "required": ["argument_1"],
        },
    }
]




In [5]:
def func_1(argument_1):
    return argument_1 * 2




In [34]:
function_schema = [
    {
        "name": "calculate_circle_area",
        "description": "Calculate the area of a circle given the radius.",
        "parameters": {
            "type": "object",
            "properties": {"radius": {"type": "number", "description": "The radius of the circle in centimeters."}},
            "required": ["radius"],
        },
    }
]



In [36]:
user_message = "Calculate the area of a circle with a radius of five meters and also calculate the area for radius 20cms."

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), api_version="2024-07-01-preview", azure_endpoint=os.getenv("OPENAI_API_HOST")
)




In [37]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": user_message}],
    functions=function_schema,
    function_call="auto",
)



In [55]:
tools = [{
        "type": "function",
        "function": {
            "name": "calculate_circle_area",
            "description": "Calculate the area of a circle given the radius.",
            "parameters": {
                "type": "object",
                "properties": {"radius": {"type": "number", "description": "The radius of the circle in centimeters."}},
                "required": ["radius"],
            }
        }
    }]

In [56]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": user_message}],
    tools=tools,
    tool_choice="auto",
)

In [41]:
response

ChatCompletion(id='chatcmpl-BxuUIWLOKHsfVnQNTrS0onyhDq4uq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LmEhylk3gM3E0PWmzP7ceJzs', function=Function(arguments='{"radius": 500}', name='calculate_circle_area'), type='function'), ChatCompletionMessageToolCall(id='call_gkKwKfyG4oQXvXDPPz4qjJZR', function=Function(arguments='{"radius": 20}', name='calculate_circle_area'), type='function')]), content_filter_results={})], created=1753618190, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_efad92c60b', usage=CompletionUsage(completion_tokens=47, prompt_tokens=80, total_tokens=127, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=

In [30]:
import pprint
pprint.pprint(response, indent=4)

ChatCompletion(id='chatcmpl-BxuNrPUYZduCHsecrlFuGfRVqmP3r', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"radius":500}', name='calculate_circle_area'), tool_calls=None), content_filter_results={})], created=1753617791, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_efad92c60b', usage=CompletionUsage(completion_tokens=16, prompt_tokens=80, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered'

In [43]:
message = response.choices[0].message
print("First message", message)



First message ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LmEhylk3gM3E0PWmzP7ceJzs', function=Function(arguments='{"radius": 500}', name='calculate_circle_area'), type='function'), ChatCompletionMessageToolCall(id='call_gkKwKfyG4oQXvXDPPz4qjJZR', function=Function(arguments='{"radius": 20}', name='calculate_circle_area'), type='function')])


In [60]:
choices = response.choices

In [61]:
choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lOwM7PSMm3OLFjQr8O5Xu9lI', function=Function(arguments='{"radius": 500}', name='calculate_circle_area'), type='function'), ChatCompletionMessageToolCall(id='call_IJ1pi19vcn0ozCaVsQ9jNYQQ', function=Function(arguments='{"radius": 20}', name='calculate_circle_area'), type='function')]), content_filter_results={})

In [45]:
message.function_call

In [62]:
input_message = [
    {"role": "user", "content": user_message},
    choices[0].message
]

In [63]:
message = choices[0].message
if hasattr(message, "tool_calls"):
    for tool in message.tool_calls:
        call_id = tool.id
        function_name = tool.function.name
        arguments = json.loads(tool.function.arguments)
        if function_name == "calculate_circle_area":
            radius = arguments.get("radius")
            area = calculate_circle_area(radius)

            # Convert the result to a string to send back to the model
            function_response = str(area)

            input_message.append(
                {
                    "role":"tool",
                    "tool_call_id": tool.id,
                    "content": function_response
                }
            )
            
            # Send the result back to the model
               


radius: 500, area: 785398.1633974483
radius: 20, area: 1256.6370614359173


In [64]:
input_message

[{'role': 'user',
  'content': 'Calculate the area of a circle with a radius of five meters and also calculate the area for radius 20cms.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lOwM7PSMm3OLFjQr8O5Xu9lI', function=Function(arguments='{"radius": 500}', name='calculate_circle_area'), type='function'), ChatCompletionMessageToolCall(id='call_IJ1pi19vcn0ozCaVsQ9jNYQQ', function=Function(arguments='{"radius": 20}', name='calculate_circle_area'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_lOwM7PSMm3OLFjQr8O5Xu9lI',
  'content': '785398.1633974483'},
 {'role': 'tool',
  'tool_call_id': 'call_IJ1pi19vcn0ozCaVsQ9jNYQQ',
  'content': '1256.6370614359173'}]

In [65]:
second_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=input_message,
                tools=tools,
                tool_choice = "auto"
            )
final_answer = second_response.choices[0].message.content
print("Message:", message)
print("final_answer:", final_answer)

Message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lOwM7PSMm3OLFjQr8O5Xu9lI', function=Function(arguments='{"radius": 500}', name='calculate_circle_area'), type='function'), ChatCompletionMessageToolCall(id='call_IJ1pi19vcn0ozCaVsQ9jNYQQ', function=Function(arguments='{"radius": 20}', name='calculate_circle_area'), type='function')])
final_answer: The area of the circle with a radius of 5 meters (500 cm) is approximately 785398.16 cm², and the area of the circle with a radius of 20 cm is approximately 1256.64 cm².


In [20]:
input_message

[{'role': 'user',
  'content': 'Calculate the area of a circle with a radius of five meters.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"radius":500}', name='calculate_circle_area'), tool_calls=None),
 {'role': 'function',
  'name': 'calculate_circle_area',
  'content': '785398.1633974483'}]

In [21]:
final_answer

'The area of a circle with a radius of five meters is approximately \\( 78.54 \\) square meters. \n\nThe formula for the area \\( A \\) of a circle is given by:\n\n\\[\nA = \\pi r^2\n\\]\n\nSubstituting the radius \\( r = 5 \\) meters:\n\n\\[\nA = \\pi (5)^2 = \\pi \\cdot 25 \\approx 78.54 \\, \\text{m}^2\n\\]'